### 첫번째 레이어
- 합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수(Activation function) relu, 맥스풀링(kernel_size=2, stride=2)

###  두번째 레이어
- 합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수(Activation function) relu, 맥스풀링(kernel_size=2, stride=2)

### 세번째 레이어
- batch_size * 7 * 7 * 64 => batch_size = 3136(특성맵을 펼치는 레이어)
- Fully connected(전결합층 뉴런 10개) + 활성화 함수 softmax

In [1]:
# 구현에 필요한 라이브러리
import torch
import torch.nn as nn


In [2]:
# 임의의 텐서 생성 (1, 1, 28, 28) => mnist data를 받을 Tensor변수
inputs = torch.Tensor(1, 1, 28, 28)
print(f"텐서의 크기: {inputs.shape}")


텐서의 크기: torch.Size([1, 1, 28, 28])


In [5]:
# 첫 번째 합성곱층 선언하기
conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) # Conv2d 내의 인자값 (input_channel, output_channel, 커널(filter?) 사이즈, 패딩)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [12]:
# 두 번쨰 합성곱층 선언
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [13]:
# 세 번째 합성곱층 선언(풀링 레이어)
pool = nn.MaxPool2d(2) # defalut값 들로 넘어간다. 아래 코드와 동일
#pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [14]:
# 선언된 레이어를 연결하는 과정
# 첫 번째 레이어 지나가기
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [15]:
# 컨벌루셔널 레이어를 거쳐 갈 경우 마지막에 pooling layer가 존재함
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [16]:
# 두 번째 레이어 지나가기
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [17]:
# pooling layer
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [18]:
# view() 함수를 사용 텐서를 펼치는 작업
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [19]:
fc = nn.Linear(3136, 10) # Fully Connected 전체를 연결한 기초적인 신경망
out = fc(out) # 통합된 3136크기의 텐서를 통해 0~9까지의 결과를 갖게되기 때문에 10개의 output
print(out.shape)

# 전체 모델 설계, 설정 과정

torch.Size([1, 10])


In [21]:
# 위에서 선언한 신경망을 통해 mnist분류 문제 해결하기

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init


In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.is_available()함수:boolean형으로 CUDA사용 가부를 리턴한다.

# gpu사용시 고정시킬 시드 선언
torch.manual_seed(777)

#  만약 cpu가 아닌 gpu를 사용해서 훈련을 진행할 경우 고정된 시들를 통해 gpu로 학습
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [26]:
# 학습에 사용되는 변수(파라미터들: 학습율, 에폭, 배치사이즈 등) 선언
learning_rate = 0.001
training_epoch = 15
batch_size = 100

In [24]:
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                        download=True)

31.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

91.5%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
# 데이터로더를 이용 배치크기를 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [42]:
class CNN(torch.nn.Module):  # CNN구조를 사용한 모델을 선언하기 위한 클래스
    def __init__(self):  # 초기화 함수, 동시에 모델의 구조를 선언하는 함수
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):  # 레이어를 거치며 계산되는 가중치를 전달하기 위해 사용하는 함수
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out
    

In [43]:
model = CNN().to(device) #model이라는 객체를 통해 CNN 클래스가 선언되고, 이는 .to(device)를 통해 gpu(or cpu)에서 연산을 한다.


In [44]:
# 비용함수(cost function = loss function)은 크로스엔트로피로 설정
# 옵티마이져는 Adam으로 설정
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [45]:
# mnist dataset을 60,000개의 데이터를 가지고 있어서, batch_size를 100번으로 설정했으므로 총 600의 total_batch를 가진다
total_batch = len(data_loader)
print(f"총 배치의 수: {total_batch}")

총 배치의 수: 600


In [46]:
for epoch in range(training_epoch):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print(f"[Epoch: {epoch+1:>4}] cost = {avg_cost:<.9}")

[Epoch:    1] cost = 0.226501957
[Epoch:    2] cost = 0.0614006966
[Epoch:    3] cost = 0.0449174941
[Epoch:    4] cost = 0.0364840403
[Epoch:    5] cost = 0.0296810493
[Epoch:    6] cost = 0.0251575243
[Epoch:    7] cost = 0.021306945
[Epoch:    8] cost = 0.0185638536
[Epoch:    9] cost = 0.0150840767
[Epoch:   10] cost = 0.0139622176
[Epoch:   11] cost = 0.0108923903
[Epoch:   12] cost = 0.00906769186
[Epoch:   13] cost = 0.00858902466
[Epoch:   14] cost = 0.0069402284
[Epoch:   15] cost = 0.00618779287


In [47]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())

Accuracy:  0.986299991607666


C:\Users\17\.conda\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\17\.conda\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
